<h4>(1 - Preferred). Create and save a new notebook to enable automatic re-mounting of drive storage. Copy and paste the below code into the new notebook and follow the directions in the <em>How To Use</em> section of the <em>README</em>: <a href="http://colab.research.google.com/#create=true" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h4><h4>(2). Open the pre-formed version of Colab (requires manual authorization each time a notebook is opened) <a href="https://colab.research.google.com/github/tdulcet/Distributed-Computing-Scripts/blob/master/google-colab/GoogleColabGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h4>

In [ ]:
#@title <strong> <h1>📔 Colab CPU Notebook</strong></h1> 
#{ output-height: 8, form-width: "96%", display-mode: "form" }

#@markdown <h4>↖️ Click the ▶ button after deciding on the options below.</h4>
#@markdown <h4>📜 See <a href="https://github.com/tdulcet/Distributed-Computing-Scripts/tree/master/google-colab" target="_blank">documentation</a> for more information.</h4>
#@markdown <h4>💡 Keep each notebook <strong>open</strong> in the browser or 📌 the tab to a <a href="https://support.mozilla.org/en-US/kb/pinned-tabs-keep-favorite-websites-open" target="_blank">dedicated window</a> to prevent disconnection.</h4>
#@markdown <h4>🕷️ A <em>debug</em> field outputs Prime95 progress, then exits.</h4>
#@markdown <h4>🤷‍♂️ Optionally, create a GIMPS/PrimeNet account <a href="https://www.mersenne.org/update/" target="_blank">here</a> and <a href="https://www.mersenne.org/jteam/" target="_blank">join</a> the "Portland State University" team!</h4>

prime_ID = 'Default' #@param ['Default'] {allow-input: true}
computer_name = 'Default' #@param ['Default'] {allow-input: true}
type_of_work = '100 - First time LL tests' #@param ['0 - Whatever makes the most sense', '100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests', '150 - First time PRP tests', '151 - Double-check PRP tests', '152 - World record PRP tests', '154 - 100 million digit PRP tests']
computer_number = "Default (1)" #@param ["Default (1)", "2"]{allow-input: true}
debug = 'False' #@param ['False', 'CPU (Prime95)']

import os

class StopExecution(Exception):
  def _render_traceback_(self):
      pass

def load_drive():
  '''Load & cd into gdrive for persistent data'''
  if os.getcwd() != "/content/drive/My Drive/GIMPS": # don't create a subfolder in GIMPS/
      !mkdir -p GIMPS
  if os.path.exists("/content/gdrive/My Drive"): # use our notebook
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd "/content/gdrive/My Drive/GIMPS"
  elif os.path.exists("/content/drive/My Drive"): # create your own notebook with our code
    %cd "/content/drive/My Drive/GIMPS"

def run():
  '''Run Mprime'''
  !cd mprime_cpu && chmod 777 mprime && ./mprime -A$computer_number -d | tee -a "{'cpu' + computer_number + '.out'}"

def debug_exit():
  '''Output of Prime95 CPU outputs'''
  if debug == 'CPU (Prime95)' and os.path.exists('mprime_cpu/cpu' + computer_number + '.out'):
    !echo -e "\nCPU (Prime95) output:\n" && tail -n 100 "{'mprime_cpu/cpu' + computer_number + '.out'}"; echo # view mprime progress
  else:
    print(f'No `{debug}` output file found for debug option and computer number `{computer_number}`.\n')

def install():
  '''Download/Install/Configure Prime95'''
  !wget https://raw.github.com/tdulcet/Linux-System-Information/master/info.sh -qO - | bash -s # Check System Info
  !wget -N https://github.com/tdulcet/Distributed-Computing-Scripts/archive/master.zip -nv
  !unzip -o master.zip
  !sed -i '/^nohup / s/^/# /' Distributed-Computing-Scripts-master/mprime2.sh # Do not start Prime95
  !sed -i '/^crontab / s/^/# /' Distributed-Computing-Scripts-master/mprime2.sh # Do not create a cronjob
  !sed -i 's/"mprime"/"mprime_cpu"/' Distributed-Computing-Scripts-master/mprime2.sh # Name the folder specific to the runtime type
  !bash -- Distributed-Computing-Scripts-master/mprime2.sh $computer_number $prime_ID $computer_name $type_of_work # Run script
  run()

load_drive()

# use/cleanup input from user
prime_ID = 'psu' if prime_ID.lower() == 'default' else prime_ID
computers_name = "" if computer_name.lower() == 'default' else computer_name
type_of_work = type_of_work.split("-")[0].rstrip()
computer_number = "1" if computer_number.lower() == 'default (1)' else computer_number
debug = False if debug == 'False' else debug

# Add quotes to string args so script can parse spaces/special characters
prime_ID = f'"'+ prime_ID + f'"'
computer_name = f'"'+ computer_name + f'"'

if debug:
  debug_exit()
  raise StopExecution

elif not computer_number.isdigit():
  print("ERROR: Computer number must be a number")
  raise StopExecution

elif os.path.exists('mprime_cpu/work000' + computer_number + '.txt'):
  run()

else:
  install()

print("Gracefully exiting...")